
# F1 Strategy ML Project — End-to-End Notebook
**Goal:** Build models for pit-stop and tyre strategy prediction using FastF1/OpenF1 and tabular & sequence models.
This notebook provides a complete pipeline and strong modelling practices (feature engineering, imbalanced handling,
Optuna hyperparameter tuning, stacking ensembles, calibration, SHAP explainability) designed to *aim* for high accuracy.
**Important:** I cannot train models or access external APIs from this environment. Run this notebook locally or in Google Colab to execute data ingestion and training.

**How to use**
1. Run Part 1 to install packages and ingest data (FastF1 or local CSVs).
2. Follow Parts 2–6 sequentially.
3. Use GPU runtime in Colab for sequence models (LSTM).
4. Target accuracy: the notebook contains techniques meant to reach high performance; actual achieved accuracy depends on data quality and hyperparameter tuning.



## Part 1 — Environment setup & Data ingestion

**What this cell does**
- Installs required packages (uncomment for Colab)
- Sets up directory structure
- Shows how to load FastF1 sessions or local CSV/parquet

**NOTE:** In Colab, uncomment the `!pip install` line. Locally, install packages in your environment.


In [5]:
!pip install fastf1 xgboost lightgbm optuna shap imbalanced-learn scikit-learn tensorflow pandas matplotlib seaborn nbformat joblib requests


  Using cached fastf1-3.6.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached xgboost-3.1.1-py3-none-win_amd64.whl.metadata (2.1 kB)
  Using cached imbalanced_learn-0.14.0-py3-none-any.whl.metadata (8.8 kB)
  Using cached tensorflow-2.20.0-cp311-cp311-win_amd64.whl.metadata (4.6 kB)
  Using cached requests_cache-1.2.1-py3-none-any.whl.metadata (9.9 kB)
  Using cached timple-0.1.8-py3-none-any.whl.metadata (2.0 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached grpcio-1.76.0-cp311-cp311-win_amd64.whl.metadata (3.8 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.12.0-py3-none-any.whl.metadata (5.9 kB)
  Using cached h5py-3.15.1-cp311-cp311-win

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\HP\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\fault_injection\\fault_injection_service_config_parser.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\HP\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import requests
import pandas as pd

# Example: Get driver laps from 2024 Bahrain GP (race ID 1)
url = "https://api.openf1.org/v1/laps?session_key=latest&driver_number=1"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame(data)
    print(df.head())
else:
    print("Error:", response.status_code)


   meeting_key  session_key  driver_number  lap_number  \
0         1273         9869              1           1   
1         1273         9869              1           2   
2         1273         9869              1           3   
3         1273         9869              1           4   
4         1273         9869              1           5   

                         date_start  duration_sector_1  duration_sector_2  \
0                              None                NaN             39.288   
1  2025-11-09T17:04:41.604000+00:00             21.320             75.859   
2  2025-11-09T17:06:41.982000+00:00             21.675             58.682   
3  2025-11-09T17:08:30.947000+00:00             25.318             59.748   
4  2025-11-09T17:10:17.731000+00:00             30.078             50.463   

   duration_sector_3  i1_speed  i2_speed  is_pit_out_lap  lap_duration  \
0             17.131     306.0     244.0           False           NaN   
1             23.234     120.0     199.0

In [2]:
import os
from pathlib import Path
DATA_DIR = Path('/content/f1_data')  # change to your preferred path (Colab default)
CACHE_DIR = Path('/content/fastf1_cache')
DATA_DIR.mkdir(parents=True, exist_ok=True)
CACHE_DIR.mkdir(parents=True, exist_ok=True)

print('DATA_DIR =', DATA_DIR)
print('CACHE_DIR =', CACHE_DIR)


DATA_DIR = \content\f1_data
CACHE_DIR = \content\fastf1_cache


In [ ]:

# FastF1 quick ingestion example. Requires internet access and fastf1 package.
# Replace YEAR, GP, SESSION as needed. If running in Colab, run the pip install cell above first.

try:
    import fastf1
    fastf1.Cache.enable_cache(str(CACHE_DIR))
    YEAR = 2023
    GP = 'Monaco'
    SESSION = 'R'  # 'FP1','FP2','Q','R'
    session = fastf1.get_session(2023, Monaco, R)
    session.load(laps=True, telemetry=False)  # telemetry can be heavy; toggle as needed
    laps = session.laps
    print(f'Loaded {len(laps)} laps from {2023} {Monaco} {R}')
    # Save to parquet for reproducibility
    laps.to_parquet(DATA_DIR / f'laps_{2023}_{Monaco}_{R}.parquet')
except Exception as e:
    print('FastF1 load failed; ensure fastf1 installed and internet access available. Error:', e)
    print('You can also provide local CSV/parquet files in DATA_DIR and skip FastF1.')


FastF1 load failed; ensure fastf1 installed and internet access available. Error: name 'Monaco' is not defined
You can also provide local CSV/parquet files in DATA_DIR and skip FastF1.



## Part 2 — Feature Engineering & Labeling (Detailed)
This section builds canonical per-lap features and task labels:
- `pit_next` (binary): whether the car pits next lap
- `next_lap_delta` (regression): next-lap time delta (proxy for tyre degradation)
- Additional recommended features: gap to leader, competitor tyre ages, ambient/track temps, stint-length, compound encodings

The code below expects a `laps` DataFrame (from FastF1 or local parquet).


In [ ]:

import numpy as np
import pandas as pd
from pathlib import Path

# load saved laps parquet (from Part 1) or local CSV
# adapt filename to your saved files
parquet_candidates = list(DATA_DIR.glob('laps_*.parquet'))
if parquet_candidates:
    laps_df = pd.read_parquet(parquet_candidates[0])
    print('Loaded', parquet_candidates[0])
else:
    raise FileNotFoundError('No laps parquet found in DATA_DIR. Run the FastF1 cell or upload data.')

# Basic cleaning and canonical features
# Keep only accurate laps
if 'IsAccurate' in laps_df.columns:
    laps_df = laps_df[laps_df['IsAccurate'] == True].copy()

# Lap time in seconds
if 'LapTime' in laps_df.columns:
    laps_df['lap_sec'] = laps_df['LapTime'].dt.total_seconds()
else:
    # fallback if lap time already float
    if 'lap_sec' not in laps_df.columns:
        raise KeyError('LapTime column not found; please ensure laps dataframe contains LapTime or lap_sec.')

# Tyre age within stint
laps_df['Stint'] = laps_df['Stint'].fillna(0).astype(int)
laps_df['tyre_age'] = laps_df.groupby(['Driver', 'Stint']).cumcount() + 1

# Lap delta and rolling features
laps_df['prev_lap_sec'] = laps_df.groupby('Driver')['lap_sec'].shift(1)
laps_df['lap_delta'] = laps_df['lap_sec'] - laps_df['prev_lap_sec']
laps_df['rolling_mean_3'] = laps_df.groupby('Driver')['lap_sec'].rolling(3, min_periods=1).mean().reset_index(0,drop=True)

# Weather join placeholder (FastF1 provides session.weather)
# If session available: session.weather['AmbientTemperature'] or session.weather['TrackTemperature']
# For local CSVs, include columns ambient_temp, track_temp per lap if available.

# Label: pit_next (needs pit stop table)
pit_next = np.zeros(len(laps_df), dtype=int)
try:
    # FastF1 pit stops may exist: session.load_pit_data()
    pitstops = session.load_pit_data()
    pits = {(r['Driver'], int(r['Lap'])) for r in pitstops.to_dict('records')}
    for i, row in laps_df.iterrows():
        if (row['Driver'], int(row['LapNumber'])+1) in pits:
            pit_next[i] = 1
    laps_df['pit_next'] = pit_next
    print('pit_next labels created; positives =', laps_df['pit_next'].sum())
except Exception as e:
    print('Pit stop data not available; create pit_next from external pit CSV or set manually. Error:', e)
    laps_df['pit_next'] = 0

# Regression target: next_lap_delta as a proxy for tyre degradation
laps_df['next_lap_sec'] = laps_df.groupby('Driver')['lap_sec'].shift(-1)
laps_df['next_lap_delta'] = laps_df['next_lap_sec'] - laps_df['lap_sec']

# Save features for modeling
FEATURES_PATH = DATA_DIR / 'features_parquet.parquet'
cols_to_save = ['Driver', 'Team', 'LapNumber', 'Stint', 'Compound', 'tyre_age', 'lap_sec', 'lap_delta',
                'rolling_mean_3', 'pit_next', 'next_lap_delta']
# Some columns may be missing depending on data; filter
cols_to_save = [c for c in cols_to_save if c in laps_df.columns]
laps_df[cols_to_save].to_parquet(FEATURES_PATH)
print('Saved features to', FEATURES_PATH)



## Part 3 — Baseline Modeling + Hyperparameter Tuning (Optuna)
We build:
- Pit-next classifier (XGBoost) with Optuna tuning and cross-validation
- Tyre degradation regressor (XGBoost) with Optuna tuning

We'll include handling class imbalance (scale_pos_weight or SMOTE) and stacking ensemble later.


In [ ]:

import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
import xgboost as xgb
import optuna
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support, mean_squared_error, mean_absolute_error
from imblearn.over_sampling import SMOTE

FEATURES_PATH = DATA_DIR / 'features_parquet.parquet'
df = pd.read_parquet(FEATURES_PATH)
df = df.dropna(subset=['lap_sec'])

# Encode categoricals
for col in ['Compound', 'Driver', 'Team']:
    if col in df.columns:
        df[col] = df[col].astype(str)
        df[col + '_enc'] = LabelEncoder().fit_transform(df[col])

# Common features
feature_cols = [c for c in ['lap_sec', 'tyre_age', 'lap_delta', 'rolling_mean_3', 'Compound_enc'] if c in df.columns or c.endswith('_enc')]
feature_cols = [c for c in df.columns if c.endswith('_enc')] + [c for c in ['lap_sec', 'tyre_age', 'lap_delta', 'rolling_mean_3'] if c in df.columns]
feature_cols = [c for c in feature_cols if c in df.columns]
print('Using features:', feature_cols)

# -------------------------
# Pit-next classifier
# -------------------------
df_clf = df.dropna(subset=['pit_next'])
X = df_clf[feature_cols].fillna(0)
y = df_clf['pit_next'].astype(int)

# Handle class imbalance using SMOTE or scale_pos_weight
print('Class distribution:', y.value_counts())

# Simple train-test split for final eval
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Option: SMOTE (careful with time-series leakage; ensure you only apply to training set)
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('After resampling, class distribution:', np.bincount(y_res))

# Optuna objective for XGBoost binary
def objective_clf(trial):
    param = {
        'verbosity': 0,
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'booster': 'gbtree',
        'tree_method': 'hist',
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_float('alpha', 1e-3, 10.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
    }
    cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
    aucs = []
    for train_idx, val_idx in cv.split(X_res, y_res):
        dtrain = xgb.DMatrix(X_res.iloc[train_idx], label=y_res[train_idx])
        dval = xgb.DMatrix(X_res.iloc[val_idx], label=y_res[val_idx])
        bst = xgb.train(param, dtrain, num_boost_round=1000, evals=[(dval,'val')], early_stopping_rounds=25, verbose_eval=False)
        pred = bst.predict(dval)
        aucs.append(roc_auc_score(y_res[val_idx], pred))
    return np.mean(aucs)

import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective_clf, n_trials=40)
print('Best params:', study.best_params)

# Train final model on full resampled train set with best params
best_params = study.best_params
best_params.update({'verbosity':0, 'objective':'binary:logistic', 'eval_metric':'auc', 'booster':'gbtree','tree_method':'hist'})
dtrain_full = xgb.DMatrix(X_res, label=y_res)
dtest_full = xgb.DMatrix(X_test, label=y_test)
bst_clf = xgb.train(best_params, dtrain_full, num_boost_round=1000, evals=[(dtest_full,'test')], early_stopping_rounds=25)

# Evaluate
probs = bst_clf.predict(dtest_full)
auc = roc_auc_score(y_test, probs)
pred = (probs > 0.5).astype(int)
print('Test AUC:', auc)
print('Precision/Recall/F1:', precision_recall_fscore_support(y_test, pred, average='binary'))

# -------------------------
# Tyre degradation regressor
# -------------------------
df_reg = df.dropna(subset=['next_lap_delta'])
Xr = df_reg[feature_cols].fillna(0)
yr = df_reg['next_lap_delta'].astype(float)

# Train-test split
Xr_train, Xr_test, yr_train, yr_test = train_test_split(Xr, yr, test_size=0.2, random_state=42)

def objective_reg(trial):
    param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'booster': 'gbtree',
        'tree_method': 'hist',
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_float('alpha', 1e-3, 10.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
    }
    cv = 4
    cv_scores = []
    kf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
    # use a simple grouping for CV: stratify by sign of yr to create folds (not perfect)
    y_sign = (yr_train > 0).astype(int)
    for train_idx, val_idx in StratifiedKFold(n_splits=4, shuffle=True, random_state=42).split(Xr_train, y_sign):
        dtr = xgb.DMatrix(Xr_train.iloc[train_idx], label=yr_train.iloc[train_idx])
        dval = xgb.DMatrix(Xr_train.iloc[val_idx], label=yr_train.iloc[val_idx])
        bst = xgb.train(param, dtr, num_boost_round=1000, evals=[(dval,'val')], early_stopping_rounds=25, verbose_eval=False)
        pred = bst.predict(dval)
        cv_scores.append(mean_squared_error(yr_train.iloc[val_idx], pred, squared=False))
    return np.mean(cv_scores)

study_r = optuna.create_study(direction='minimize')
study_r.optimize(objective_reg, n_trials=40)
print('Best reg params:', study_r.best_params)

# Train final regressor
best_r = study_r.best_params
best_r.update({'verbosity':0, 'objective':'reg:squarederror', 'eval_metric':'rmse', 'booster':'gbtree','tree_method':'hist'})
dtrain_r = xgb.DMatrix(Xr_train, label=yr_train)
dtest_r = xgb.DMatrix(Xr_test, label=yr_test)
bst_reg = xgb.train(best_r, dtrain_r, num_boost_round=1000, evals=[(dtest_r,'test')], early_stopping_rounds=25)
yr_pred = bst_reg.predict(dtest_r)
print('Reg RMSE:', mean_squared_error(yr_test, yr_pred, squared=False))
print('Reg MAE:', mean_absolute_error(yr_test, yr_pred))

# Save models
import joblib
MODEL_DIR = DATA_DIR / 'models'
MODEL_DIR.mkdir(exist_ok=True)
bst_clf.save_model(str(MODEL_DIR / 'xgb_pit_clf.json'))
bst_reg.save_model(str(MODEL_DIR / 'xgb_degradation.json'))
print('Models saved to', MODEL_DIR)



## Part 4 — Sequence models (LSTM/GRU) and Simulator Integration
This section trains an LSTM to forecast next-lap times using sequences of past lap times and other features.
Then we use the learned predictor inside a Monte-Carlo simulator to evaluate candidate pit strategies.


In [ ]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import StandardScaler

# Build sequences per stint
FEATURES_PATH = DATA_DIR / 'features_parquet.parquet'
df = pd.read_parquet(FEATURES_PATH)

# We'll create sequences of lap_sec per stint, with optional compound & tyre age features
SEQ_LEN = 20
seqs = []
targets = []
meta = []
for (driver, stint), g in df.groupby(['Driver','Stint']):
    g = g.sort_values('LapNumber')
    arr = g['lap_sec'].values
    if len(arr) < 3:
        continue
    for i in range(1, len(arr)):
        start = max(0, i-SEQ_LEN)
        seq = arr[start:i]
        if len(seq) < SEQ_LEN:
            seq = np.pad(seq, (SEQ_LEN-len(seq), 0), 'constant', constant_values=0)
        seqs.append(seq)
        targets.append(arr[i])  # predict absolute next lap time
        meta.append({'driver': driver, 'stint': stint})
X = np.array(seqs).reshape(-1, SEQ_LEN, 1)
y = np.array(targets)

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

# Scale data per feature
sc = StandardScaler()
# fit on flattened train
X_train_flat = X_train.reshape(-1, X_train.shape[-1])
sc.fit(X_train_flat)
def scale_X(x):
    flat = x.reshape(-1, x.shape[-1])
    flat_s = sc.transform(flat)
    return flat_s.reshape(x.shape)

X_train_s = scale_X(X_train)
X_val_s = scale_X(X_val)

# LSTM model
model = Sequential([
    Masking(mask_value=0., input_shape=(SEQ_LEN,1)),
    LSTM(128, return_sequences=False),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
es = EarlyStopping(patience=6, restore_best_weights=True)
mc = ModelCheckpoint(str(DATA_DIR / 'models' / 'lstm_best.h5'), save_best_only=True, monitor='val_loss')
model.fit(X_train_s, y_train, validation_data=(X_val_s, y_val), epochs=50, batch_size=128, callbacks=[es, mc])

# Example: integrate into simple simulator
def predict_next_lap_time_from_sequence(last_seq):
    # last_seq should be array of length SEQ_LEN
    arr = np.array(last_seq).reshape(1, SEQ_LEN, 1)
    arr_s = scale_X(arr)
    return float(model.predict(arr_s)[0,0])

# Simple Monte Carlo simulator for candidate pit laps
def monte_carlo_simulate(current_seq, current_lap, remaining_laps, pit_candidates, pit_loss=22.0, trials=50):
    results = {p: [] for p in pit_candidates}
    for t in range(trials):
        for p in pit_candidates:
            seq = list(current_seq)
            total = 0.0
            lap = current_lap
            # simulate until pit
            while lap < p:
                pred = predict_next_lap_time_from_sequence(seq[-SEQ_LEN:])
                total += pred
                seq.append(pred)
                lap += 1
            # pit loss
            total += pit_loss
            # after pit, assume fresh tyre with faster times (simplified)
            for lap_after in range(lap, current_lap + remaining_laps + 1):
                # use model with reset sequence (e.g., duplicated last few fresh laps)
                pred = predict_next_lap_time_from_sequence(seq[-SEQ_LEN:])
                total += pred
                seq.append(pred)
            results[p].append(total)
    # return expected total times per candidate
    return {p: np.mean(results[p]) for p in pit_candidates}

# Example usage (requires a `current_seq` of recent lap times)
print('LSTM training complete; use monte_carlo_simulate to compare candidate pit laps.')



## Part 5 — Explainability (SHAP) & Calibration
Use SHAP to explain tree-based models (XGBoost). For classifier probability calibration, use isotonic or Platt scaling.


In [ ]:

import shap
import numpy as np
import joblib
from sklearn.calibration import CalibratedClassifierCV

# Load model (example path)
MODEL_DIR = DATA_DIR / 'models'
clf_path = MODEL_DIR / 'xgb_pit_clf.json'
if clf_path.exists():
    bst = xgb.Booster()
    bst.load_model(str(clf_path))
    # Wrap as sklearn estimator for SHAP TreeExplainer convenience
    # Use shap.TreeExplainer on xgboost booster directly
    # To compute SHAP values:
    dtest = xgb.DMatrix(X_test)
    explainer = shap.TreeExplainer(bst)
    shap_values = explainer.shap_values(dtest)
    # summary
    try:
        shap.summary_plot(shap_values, X_test)
    except Exception as e:
        print('SHAP plotting may require graphical backend (use Colab/Local). Error:', e)
else:
    print('Classifier model not found; run Part 3 to train and save models.')

# Probability calibration (Platt scaling example)
# If you trained an XGBClassifier via sklearn API, use CalibratedClassifierCV
# clf_sklearn = xgb.XGBClassifier(**best_params)
# clf_cal = CalibratedClassifierCV(clf_sklearn, cv=3, method='isotonic')



## Part 6 — Deployment, CI, and Next Steps
This section outlines how to:
- Serve model via FastAPI + Docker
- Create a real-time ingestion pipeline (OpenF1 WebSocket / FastF1 caching)
- Continuous evaluation and backtesting: rolling windows, leaderboard, and benchmark scripts.

Also includes a checklist for improving performance to approach target accuracy:
- Use cross-season training and feature-store with more tracks/seasons
- Augment features: telemetry-derived corner/straight speeds, DRS usage, fuel proxies
- Ensembling (stacking multiple models) and careful leakage-free CV
- Hyperparameter budget increase (Optuna 200+ trials)
- Domain adaptation / transfer learning for new circuits


In [ ]:

# Minimal FastAPI app scaffold (save as app.py)
fastapi_app = r"""
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import numpy as np

app = FastAPI()

class State(BaseModel):
    driver: str
    lap_number: int
    tyre_age: int
    lap_sec: float
    compound: str

# load model artifacts
# clf = joblib.load('models/pit_clf.joblib')

@app.post('/pit_reco')
def pit_reco(state: State):
    # implement featurization and model prediction here
    return {'prob_pit_next': 0.12, 'explanation': {'tyre_age': 0.4, 'lap_delta': -0.1}}
"""

with open('/content/fastapi_example.py', 'w') as f:
    f.write(fastapi_app)
print('FastAPI scaffold written to /content/fastapi_example.py (adjust paths and load models).')
